<a href="https://colab.research.google.com/github/aruntakhur/PyTorch_NLP_Vision_New/blob/main/Muril_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 78.8 MB/s 
     |████████████████████████████████| 895 kB 74.6 MB/s 
     |████████████████████████████████| 596 kB 90.4 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
import json

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/DataSet_NLP/train.csv")

In [ ]:
ds_h = train[train['language']=='hindi']

In [ ]:
examples = ds_h.reset_index(drop="True")

In [ ]:
examples

,id,context,question,answer_text,answer_start,language
0,416091aeb,विषाणु अकोशिकीय अतिसूक्ष्म जीव हैं जो केवल जीव...,सन १८८६ में किसने बताया कि तम्बाकू में मोजेक र...,एडोल्फ मेयर,935,hindi
1,9d274ae3c,फ्लोरीन एक रासायनिक तत्व है। यह आवर्त सारणी (p...,फ्लोरीन की परमाणु संख्या क्या है?,9,166,hindi
2,da7397c5e,सीऐटल (अंग्रेजी: Seattle) अमेरिका के वाशिंगटन ...,सीटल शहर कहाँ स्थित है?,अमेरिका के वाशिंगटन राज्य,26,hindi
3,661880e43,सूर्य अथवा सूरज सौरमंडल के केन्द्र में स्थित ए...,पृथ्वी को सूर्य की परिक्रमा करने में कितने दिन...,28 दिनों,2762,hindi
4,3e3a2bed4,"दिल्ली नगर निगम एक शहर व नगर निगम है, जो दिल्ल...","दिल्ली नगर निगम, दिल्ली के कितने जिलों में कार...",कुल नौ जिलों,51,hindi
...,...,...,...,...,...,...
741,26f356026,स्वामी निगमानन्द परमहंस (18 अगस्त 1880 - 29 नव...,स्वामी निगमानन्द परमहंस के तन्त्र गुरु कौन थे?,बामाक्षेपा,2691,hindi
742,31179f1bb,भरत मुनि ने नाट्यशास्त्र नामक प्रसिद्ध ग्रन्थ ...,नित्यशास्त्र किसने लिखा है?,भरत मुनि,0,hindi
743,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi
744,7f997884d,"जलाल उद्दीन मोहम्मद अकबर () (१५ अक्तूबर, १५४२-...",मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,"२७ अक्तूबर, १६०५",46,hindi


In [ ]:
model_checkpoint = 'google/muril-large-cased'
batch_size = 8
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_length = 384
doc_stride = 192
pad_on_right = tokenizer.padding_side == "right"
n_folds = 3

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/DataSet_NLP/train.csv")
threeds = pd.read_csv('/content/drive/MyDrive/DataSet_NLP/chaii-mlqa-xquad-5folds.csv.zip')

In [ ]:
hindi = threeds[threeds.src != 'chaii'].reset_index(drop=True)

In [ ]:
train.head()

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


In [ ]:
chaii = threeds[threeds.src == 'chaii'].reset_index(drop=True)
chaii.head()

,context,question,answer_text,answer_start,language,src,fold
0,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,chaii,1
1,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,chaii,3
2,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,chaii,0
3,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,chaii,0
4,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,chaii,1


In [ ]:
chaii = pd.merge(chaii, train[['id', 'context', 'question']], how='left', on=['context', 'question'])
chaii

,context,question,answer_text,answer_start,language,src,fold,id
0,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,chaii,1,903deec17
1,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,chaii,3,d9841668c
2,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,chaii,0,29d154b56
3,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,chaii,0,41660850a
4,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,chaii,1,b29c82c22
...,...,...,...,...,...,...,...,...
1107,स्वामी निगमानन्द परमहंस (18 अगस्त 1880 - 29 नव...,स्वामी निगमानन्द परमहंस के तन्त्र गुरु कौन थे?,बामाक्षेपा,2691,hindi,chaii,3,26f356026
1108,भरत मुनि ने नाट्यशास्त्र नामक प्रसिद्ध ग्रन्थ ...,नित्यशास्त्र किसने लिखा है?,भरत मुनि,0,hindi,chaii,2,31179f1bb
1109,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi,chaii,1,0d35dc007
1110,"जलाल उद्दीन मोहम्मद अकबर () (१५ अक्तूबर, १५४२-...",मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,"२७ अक्तूबर, १६०५",46,hindi,chaii,0,7f997884d


In [ ]:
del chaii['fold']

In [ ]:
from sklearn import model_selection 
def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}

In [ ]:
chaii = create_folds(chaii, 3)

In [ ]:
chaii

,context,question,answer_text,answer_start,language,src,id,kfold
0,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,chaii,903deec17,2
1,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,chaii,d9841668c,0
2,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,chaii,29d154b56,2
3,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,chaii,41660850a,2
4,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,chaii,b29c82c22,2
...,...,...,...,...,...,...,...,...
1107,स्वामी निगमानन्द परमहंस (18 अगस्त 1880 - 29 नव...,स्वामी निगमानन्द परमहंस के तन्त्र गुरु कौन थे?,बामाक्षेपा,2691,hindi,chaii,26f356026,1
1108,भरत मुनि ने नाट्यशास्त्र नामक प्रसिद्ध ग्रन्थ ...,नित्यशास्त्र किसने लिखा है?,भरत मुनि,0,hindi,chaii,31179f1bb,2
1109,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi,chaii,0d35dc007,1
1110,"जलाल उद्दीन मोहम्मद अकबर () (१५ अक्तूबर, १५४२-...",मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,"२७ अक्तूबर, १६०५",46,hindi,chaii,7f997884d,2


In [ ]:
fold = 1
chaii_train = chaii[chaii.kfold != fold]
chaii_valid = chaii[chaii.kfold == fold]

In [ ]:
chaii_train["answers"] = chaii_train[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
chaii_valid["answers"] = chaii_valid[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
hindi["answers"] = hindi[["answer_start", "answer_text"]].apply(convert_answers, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
chaii_train["answers"]

0                 {'answer_start': [53], 'text': ['206']}
1        {'answer_start': [2358], 'text': ['காசுமீரில்']}
2       {'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட...
3            {'answer_start': [68], 'text': ['தாலாட்டு']}
4          {'answer_start': [585], 'text': ['சூரியனும்']}
                              ...                        
1097     {'answer_start': [289], 'text': ['ऑस्ट्रेलिया']}
1102    {'answer_start': [34], 'text': ['11 फरवरी 1977']}
1108          {'answer_start': [0], 'text': ['भरत मुनि']}
1110    {'answer_start': [46], 'text': ['२७ अक्तूबर, १...
1111         {'answer_start': [2017], 'text': ['नेताजी']}
Name: answers, Length: 741, dtype: object

In [ ]:
!pip install -q datasets

     |████████████████████████████████| 306 kB 8.4 MB/s 
     |████████████████████████████████| 1.1 MB 64.0 MB/s 
     |████████████████████████████████| 133 kB 79.1 MB/s 
     |████████████████████████████████| 243 kB 77.5 MB/s 
     |████████████████████████████████| 160 kB 95.5 MB/s 
     |████████████████████████████████| 271 kB 83.8 MB/s 
     |████████████████████████████████| 192 kB 80.4 MB/s 


In [ ]:
from datasets import Dataset
chaii_train_ds = Dataset.from_pandas(chaii_train)
chaii_valid_ds = Dataset.from_pandas(chaii_valid)

In [ ]:
chaii_train_ds

Dataset({
    features: ['context', 'question', 'answer_text', 'answer_start', 'language', 'src', 'id', 'kfold', 'answers', '__index_level_0__'],
    num_rows: 741
})

In [ ]:
tokenized_chaii_train_strid = chaii_train_ds.map(prepare_train_features, batched=True, remove_columns=chaii_train_ds.column_names)
tokenized_chaii_valid_strid = chaii_valid_ds.map(prepare_train_features, batched=True, remove_columns=chaii_valid_ds.column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_chaii_train_strid

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 9805
})

In [ ]:
import random
def negative_sampling(examples, ratio=0.1):
    
    def _sample(pos):
        if pos != 0: return True
        else: return random.random() < ratio
                
    indices = [i for i,x in enumerate(examples['start_positions']) if _sample(x)]

    for key in examples.keys():
        examples[key] = [x for i,x in enumerate(examples[key]) if i in indices]
        
    return examples

In [ ]:
tokenized_chaii_train_strid1 = tokenized_chaii_train_strid.map(negative_sampling, fn_kwargs={'ratio':0.1}, batched=True, batch_size=8)
tokenized_chaii_train_strid2 = tokenized_chaii_train_strid.map(negative_sampling, fn_kwargs={'ratio':0.2}, batched=True, batch_size=8)

  0%|          | 0/1226 [00:00<?, ?ba/s]

  0%|          | 0/1226 [00:00<?, ?ba/s]

In [ ]:
tokenized_chaii_train_strid1

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 1857
})

In [ ]:
from torch.utils.data import DataLoader, Dataset, IterableDataset, RandomSampler, SequentialSampler

from transformers.file_utils import is_datasets_available
from transformers import Trainer, TrainingArguments, Trainer

class MyTrainer(Trainer): 

    def get_train_dataloader(self) -> DataLoader:
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        if is_datasets_available() and isinstance(train_dataset, Dataset):
            train_dataset = self._remove_unused_columns(train_dataset, description="training")

        train_sampler = SequentialSampler(self.train_dataset)

        return DataLoader(
            train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-large-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the 

In [ ]:
EXP_NAME = 'pretraining_MURIL_71'
FOLDER_NAME = EXP_NAME
SEED = 68

In [ ]:
batch_size = 12

In [ ]:
from transformers import default_data_collator
args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    #logging_strategy = "steps",
    #logging_steps = 1000,
    #save_steps = 1000,
    #save_strategy = "steps",
    learning_rate=3e-5,
    gradient_accumulation_steps=1,
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = MyTrainer(
    model,
    args,
    train_dataset=tokenized_chaii_train_strid1,
    eval_dataset=tokenized_chaii_valid_strid,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1857
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 155


RuntimeError: ignored

In [ ]:
!sim-nvidia

/bin/bash: sim-nvidia: command not found
